In [14]:
from pathlib import Path
from dotenv import load_dotenv
import os, sys

try:
    PROJECT_ROOT = Path(__file__).resolve().parent.parent
except NameError:
    # Không có __file__ → đang ở notebook / REPL
    PROJECT_ROOT = Path.cwd().parent  # điều chỉnh số .parent nếu cần

ENV_PATH = PROJECT_ROOT / ".env"
load_dotenv(ENV_PATH)


True

In [37]:
from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import init_chat_model
from langchain_community.cache import RedisSemanticCache
import json
from langchain_core.outputs import Generation
from langfuse import observe, get_client
langfuse = get_client()
from langchain_openai import ChatOpenAI
from langfuse.langchain import CallbackHandler

langfuse_handler = CallbackHandler()

REDIS_URI: str = "localhost:6378"
embeddings = OpenAIEmbeddings(model="text-embedding-nomic-embed-text-v1.5", openai_api_base="http://localhost:4000", openai_api_key="sk-llmops")
llm_chat = ChatOpenAI(
    model="qwen-instruct",
    api_key="sk-llmops",
    temperature=0.7,
    base_url="http://localhost:4000"
  )
def fn(question, llm_chat):
  response = llm_chat.invoke(question,{
        "callbacks": [langfuse_handler],
        "metadata": {                       # trace attributes
            "langfuse_session_id": "session-1",
            "langfuse_user_id": "user-1",
        },
    },)
  return response

question = "Hello, what your name?"

response = fn(question, llm_chat)
response_dict = response.model_dump()
print(json.dumps(response_dict, indent=2, ensure_ascii=False))  

{
  "content": "I am Qwen, an AI language model developed by Alibaba Cloud. How may I assist you today?",
  "additional_kwargs": {
    "refusal": null
  },
  "response_metadata": {
    "token_usage": {
      "completion_tokens": 21,
      "prompt_tokens": 35,
      "total_tokens": 56,
      "completion_tokens_details": null,
      "prompt_tokens_details": null
    },
    "model_name": "lm_studio/qwen2.5-1.5b-instruct",
    "system_fingerprint": "qwen2.5-1.5b-instruct",
    "id": "chatcmpl-rru61vgq69qa6yoobxht5c",
    "service_tier": null,
    "finish_reason": "stop",
    "logprobs": null
  },
  "type": "ai",
  "name": null,
  "id": "run--37f8c6bb-0819-4c4f-8c14-816da4350366-0",
  "example": false,
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 35,
    "output_tokens": 21,
    "total_tokens": 56,
    "input_token_details": {},
    "output_token_details": {}
  }
}


In [ ]:
semantic_cache = RedisSemanticCache(
            redis_url="redis://localhost:6378",
            embedding=embeddings
)   

In [17]:
question = "Hello, what your name?"

# Check if the question is already cached
cached_response = semantic_cache.lookup(question, 'pre-cache')
if cached_response:
    print(cached_response[0].text)
else:
    response = fn(question, llm_chat)
    response_dict = response.model_dump()
    print(json.dumps(response_dict, indent=2, ensure_ascii=False))

    semantic_cache.update(question, 'pre-cache', [
    Generation(text=str(response.content))])


I am Qwen, a large language model developed by Alibaba Cloud. My name is Qwen, and I was created to assist you with various tasks like answering questions, providing information, and even generating creative content. How can I help you today?
